#### CartItem ETL

In [1]:
# ETL BrandDim
from jobs.extract.extracts import extract_mongo_brands_to_df
from jobs.load.loads import load_brand_df_to_mssql

brand_df = extract_mongo_brands_to_df()
load_brand_df_to_mssql(brand_df)



2024-06-07 18:06:47,294 - INFO - Connected to MongoDB database
2024-06-07 18:06:47,296 - INFO - {"message": "Waiting for suitable server to become available", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 6662e9c7afbea97a38e960d5, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('ac-wtfo0ij-shard-00-00.fnittpl.mongodb.net', 27017) server_type: Unknown, rtt: None>, <ServerDescription ('ac-wtfo0ij-shard-00-01.fnittpl.mongodb.net', 27017) server_type: Unknown, rtt: None>, <ServerDescription ('ac-wtfo0ij-shard-00-02.fnittpl.mongodb.net', 27017) server_type: Unknown, rtt: None>]>", "clientId": {"$oid": "6662e9c7afbea97a38e960d5"}, "remainingTimeMS": 30}
2024-06-07 18:06:48,848 - INFO - Connection to MongoDB closed.
2024-06-07 18:06:49,029 - INFO - Connected to MSSQL database
d:\Bi\BiDocument\HK1_2024\TLCN\eShopAnalysis\TLCN-eShopAnalysis\eShopAnalysis.ETL\eTLCartItemDW\jobs\load\loads.py:19: UserWarning: pandas only supports 

In [ ]:
# ETL CatalogDim
import os
import pandas as pd
from jobs.extract.extracts import extract_mongo_catalogs_to_df
from jobs.load.loads import load_catalog_df_to_mssql

category_df = extract_mongo_catalogs_to_df()
#TODO:
# 1. TRANSFORM DELETE NULL CATALOG NAME AND NOT SAVE TO MSSQL
load_catalog_df_to_mssql(category_df)


In [ ]:
# ETL SubCatalogDim
from jobs.extract.extracts import extract_mongo_subcatalogs_to_df
from jobs.load.loads import load_subcatalog_df_to_mssql

subcatalog_df = extract_mongo_subcatalogs_to_df()
print(subcatalog_df)
load_subcatalog_df_to_mssql(subcatalog_df)


In [4]:
from jobs.extract.extracts import extract_mongo_usage_instruction_to_df

usage_instruction_df = extract_mongo_usage_instruction_to_df()
usage_instruction_df

2024-06-07 17:42:07,778 - INFO - Connected to MongoDB database
2024-06-07 17:42:07,779 - INFO - {"message": "Waiting for suitable server to become available", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 6662e3ff58db827f72c56fea, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('ac-wtfo0ij-shard-00-00.fnittpl.mongodb.net', 27017) server_type: Unknown, rtt: None>, <ServerDescription ('ac-wtfo0ij-shard-00-01.fnittpl.mongodb.net', 27017) server_type: Unknown, rtt: None>, <ServerDescription ('ac-wtfo0ij-shard-00-02.fnittpl.mongodb.net', 27017) server_type: Unknown, rtt: None>]>", "clientId": {"$oid": "6662e3ff58db827f72c56fea"}, "remainingTimeMS": 30}
2024-06-07 17:42:09,214 - INFO - Connection to MongoDB closed.


,ProductUsageInstruction
0,Eat right away or process into other dishes
1,None
2,"Defrost outside of the plastic bag at ambient,..."
3,Ready to eat in just three minutes
4,Ready to eat
5,Use directly
6,ready to eat
7,"Crispy and flaky cookies, perfect with cheese"
8,Serve with bread for breakfast.\nCheddar chees...
9,Mix with a snack or sandwich for a new delecta...


In [5]:
# ETL PreserveInstructionTypeDim using SpaCy and Vector Similarity
import pandas as pd

usage_instruction_df = pd.read_csv('resources/product_usage_instruction.csv')
usage_instruction_df.dropna(inplace=True, subset=['ProductUsageInstruction'])

# another approach is to use topic modeling to find the most important words in the usage instruction, 
# the most important words will be the labels for the usage instruction type
from jobs.transform.usage_instruction_transform import *
sublists_list = transform_preprocess_usage_instruction_df_to_sublists_tokens(usage_instruction_df)
lemm_stemm_sublists_list = transform_lemm_stem_sublists(sublists_list)
reduced_usage_instruction_type_df = transform_sublists_to_reduced_usage_instruction_df(lemm_stemm_sublists_list)
# print(reduced_usage_instruction_type_df)
from jobs.load.loads import load_usage_instruction_df_to_mssql
load_usage_instruction_df_to_mssql(reduced_usage_instruction_type_df)

2024-06-07 18:07:48,602 - INFO - Connected to MSSQL database
d:\Bi\BiDocument\HK1_2024\TLCN\eShopAnalysis\TLCN-eShopAnalysis\eShopAnalysis.ETL\eTLCartItemDW\jobs\load\loads.py:77: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_usage_instruction = pd.read_sql_query(query, mssql_cursor.connection)
2024-06-07 18:07:48,621 - INFO - Connection to MSSQL closed.


Usage instruction type dict:  {0: 'no usage instruction', 1: 'too special usage instruction with steps', 2: 'readi to eat', 3: 'drink directli'} 

   UsageInstructionTypeId                  UsageInstructionTypeName
0                       0                      no usage instruction
1                       1  too special usage instruction with steps
2                       2                              readi to eat
3                       3                            drink directli


In [6]:
# ETL UsageInstructionTypeDim using LDA topic modeling
import pandas as pd

usage_instruction_df = pd.read_csv('resources/product_usage_instruction.csv')
usage_instruction_df.dropna(inplace=True, subset=['ProductUsageInstruction'])

from jobs.transform.usage_instruction_transform import transform_to_usage_instruction_df_with_topic_and_word_representation
from jobs.load.loads import load_usage_instruction_df_to_mssql
# TODO create a util func to get the best number of topic
usage_instruction_df_with_topic_and_wp = transfor   m_to_usage_instruction_df_with_topic_and_word_representation(usage_instruction_df, 8)
print(usage_instruction_df_with_topic_and_wp)
# how to save the model to evaluate the new data: https://stackoverflow.com/a/22034166
# how to use this model and evaluate the new data: https://datascience.stackexchange.com/a/107383
# another document with all the above: https://medium.com/@rayhantithokharisma/latent-dirichlet-allocation-topic-modelling-with-online-learning-feature-5051ca9df749
load_usage_instruction_df_to_mssql(usage_instruction_df_with_topic_and_wp)
    

2024-06-07 17:42:30,132 - INFO - adding document #0 to Dictionary<0 unique tokens: []>
2024-06-07 17:42:30,133 - INFO - built Dictionary<197 unique tokens: ['away', 'dish', 'eat', 'process', 'right']...> from 51 documents (total 340 corpus positions)
2024-06-07 17:42:30,133 - INFO - Dictionary lifecycle event {'msg': "built Dictionary<197 unique tokens: ['away', 'dish', 'eat', 'process', 'right']...> from 51 documents (total 340 corpus positions)", 'datetime': '2024-06-07T17:42:30.133473', 'gensim': '4.2.0', 'python': '3.9.6 (tags/v3.9.6:db3ff76, Jun 28 2021, 15:26:21) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22631-SP0', 'event': 'created'}
2024-06-07 17:42:30,134 - INFO - discarding 172 tokens: [('away', 1), ('right', 1), ('ambient', 1), ('bag', 1), ('bake', 1), ('defrost', 1), ('heat', 1), ('min', 1), ('outside', 1), ('oven', 1)]...
2024-06-07 17:42:30,135 - INFO - keeping 25 tokens which were in no less than 3 and no more than 25 (=50.0%) documents
2024-06-07 17:42

In [6]:
# ETL PreserveInstructionTypeDim
import pandas as pd
from jobs.extract.extracts import extract_mongo_preserve_instruction_to_df
from jobs.transform.preserve_instruction_transform import transform_to_preserve_instruction_df_with_topic_and_word_representation
# TODO create a util func to get the best number of topic
preserve_instruction_df = extract_mongo_preserve_instruction_to_df()
preserve_instruction_df_with_topic_and_wp = transform_to_preserve_instruction_df_with_topic_and_word_representation(preserve_instruction_df, 3)
print(preserve_instruction_df_with_topic_and_wp)
from jobs.load.loads import load_preserve_instruction_df_to_mssql
load_preserve_instruction_df_to_mssql(preserve_instruction_df_with_topic_and_wp)

2024-06-07 18:08:01,458 - INFO - Connected to MongoDB database
2024-06-07 18:08:01,459 - INFO - {"message": "Waiting for suitable server to become available", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 6662ea11afbea97a38e960d9, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('ac-wtfo0ij-shard-00-00.fnittpl.mongodb.net', 27017) server_type: Unknown, rtt: None>, <ServerDescription ('ac-wtfo0ij-shard-00-01.fnittpl.mongodb.net', 27017) server_type: Unknown, rtt: None>, <ServerDescription ('ac-wtfo0ij-shard-00-02.fnittpl.mongodb.net', 27017) server_type: Unknown, rtt: None>]>", "clientId": {"$oid": "6662ea11afbea97a38e960d9"}, "remainingTimeMS": 30}
2024-06-07 18:08:02,899 - INFO - Connection to MongoDB closed.


['refrigerator', 'preferably', 'degree']
[]
[]
[]
[]
['c']
['away', 'direct', 'sunlight', 'heat', 'humidity']
['preserve', 'dry', 'cool', 'place']
['cool', 'place', 'away', 'sunlight']
['preserve', 'dry', 'cool', 'place']
['cool', 'dry', 'place', 'away', 'direct', 'sunlight']
['away', 'direct', 'light', 'cool', 'dry', 'place']
['cool', 'place', 'away', 'sunlight']
['refrigerated', 'degree', 'celsius']
['refrigerated', 'degree', 'celsius']
['refrigerated', 'low']
['cool', 'place', 'away', 'sunlight']
['away', 'heat', 'humidity']
['dry', 'cool', 'place', 'avoid', 'direct', 'contact', 'sunlight', 'well', 'date', 'open', 'refrigerated', 'degree', 'consume', 'day']
['clean', 'dry', 'place', 'away', 'rain', 'sun', 'humidity', 'leave', 'good', 'strange', 'odor', 'open', 'refrigerated', 'day']
['dry', 'cool', 'place', 'avoid', 'direct', 'contact', 'sunlight', 'well', 'date', 'open', 'refrigerated', 'degree', 'consume', 'day']
['storage', 'temperature']
['storage', 'temperature']
[]
[]
[]
['fre

2024-06-07 18:08:03,796 - INFO - adding document #0 to Dictionary<0 unique tokens: []>
2024-06-07 18:08:03,797 - INFO - built Dictionary<84 unique tokens: ['degree', 'preferably', 'refrigerator', 'c', 'away']...> from 147 documents (total 545 corpus positions)
2024-06-07 18:08:03,798 - INFO - Dictionary lifecycle event {'msg': "built Dictionary<84 unique tokens: ['degree', 'preferably', 'refrigerator', 'c', 'away']...> from 147 documents (total 545 corpus positions)", 'datetime': '2024-06-07T18:08:03.798008', 'gensim': '4.2.0', 'python': '3.9.6 (tags/v3.9.6:db3ff76, Jun 28 2021, 15:26:21) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22631-SP0', 'event': 'created'}
2024-06-07 18:08:03,798 - INFO - discarding 46 tokens: [('light', 2), ('low', 1), ('contact', 2), ('date', 2), ('well', 2), ('good', 1), ('leave', 1), ('rain', 1), ('strange', 1), ('bellow', 1)]...
2024-06-07 18:08:03,798 - INFO - keeping 38 tokens which were in no less than 3 and no more than 73 (=50.0%) docume

Topic #0: avoid, refrigerator, degree, crush, force
Topic #1: degree, open, preferably, refrigerator, damage
Topic #2: cool, place, dry, away, sunlight
{-1: 'No topic or cannot be classified', 0: 'avoid, refrigerator, degree, crush, force', 1: 'degree, open, preferably, refrigerator, damage', 2: 'cool, place, dry, away, sunlight'}
                       PreserveInstructionTypeName
-1                No topic or cannot be classified
 0       avoid, refrigerator, degree, crush, force
 1  degree, open, preferably, refrigerator, damage
 2                cool, place, dry, away, sunlight
   PreserveInstructionTypeId                     PreserveInstructionTypeName
0                         -1                No topic or cannot be classified
1                          0       avoid, refrigerator, degree, crush, force
2                          1  degree, open, preferably, refrigerator, damage
3                          2                cool, place, dry, away, sunlight


In [7]:
import pandas as pd
from jobs.extract.extracts import extract_mongo_product_to_df
from jobs.transform.product_product_models_transform import transform_product_df_to_product_df_with_fkey
from jobs.load.loads import load_product_df_to_mssql
product_not_transformed_df = extract_mongo_product_to_df()
product_df_with_dim_keys = transform_product_df_to_product_df_with_fkey(product_not_transformed_df)
load_product_df_to_mssql(product_df_with_dim_keys)

2024-06-07 18:08:10,501 - INFO - Connected to MongoDB database
2024-06-07 18:08:10,502 - INFO - {"message": "Waiting for suitable server to become available", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 6662ea1aafbea97a38e960da, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('ac-wtfo0ij-shard-00-00.fnittpl.mongodb.net', 27017) server_type: Unknown, rtt: None>, <ServerDescription ('ac-wtfo0ij-shard-00-01.fnittpl.mongodb.net', 27017) server_type: Unknown, rtt: None>, <ServerDescription ('ac-wtfo0ij-shard-00-02.fnittpl.mongodb.net', 27017) server_type: Unknown, rtt: None>]>", "clientId": {"$oid": "6662ea1aafbea97a38e960da"}, "remainingTimeMS": 30}
2024-06-07 18:08:11,945 - INFO - Connection to MongoDB closed.
2024-06-07 18:08:11,949 - INFO - Connected to MSSQL database
d:\Bi\BiDocument\HK1_2024\TLCN\eShopAnalysis\TLCN-eShopAnalysis\eShopAnalysis.ETL\eTLCartItemDW\jobs\utils\fkey_utils.py:38: UserWarning: pandas only sup

                                ProductId  \
0    8CD2075C-9574-4C4B-AA17-9E1759060773   
1    C001CBCB-AC0A-42BC-8067-22A89584D4E5   
2    21D32F7D-59F2-48EB-B526-954D66679271   
3    9682B122-93BB-4D57-AA3D-BE7FE41A9276   
4    BBFCFA77-730F-493E-8F9A-C1EF47A19EDA   
..                                    ...   
142  E0ADB939-BC77-4ACF-8D23-1720C657FDDB   
143  01091559-4883-4F19-8F25-D2F383654822   
144  A325ED0C-73D1-4857-B48D-E0F22FCCCEA2   
145  420C793F-2CF2-409F-A9FB-2DCF22D28CE7   
146  CF78719D-F3DA-49EB-83D5-1629AB1C6CDA   

                                         ProductName  BrandKey  \
0                          Vfood Chicken Eggs (6pcs)         0   
1                        Nivea Essential Care (4.8g)         1   
2                               333 Beer Can (330ml)         2   
3               Nivea Fruity Shine Strawberry (4.8g)         1   
4                  Minh Ha Vietnamese Pho Kit (138g)         3   
..                                               ...       ... 

In [8]:
from jobs.extract.extracts import extract_mongo_product_model_to_df
from jobs.load.loads import load_product_model_df_to_mssql
product_model_df = extract_mongo_product_model_to_df()
load_product_model_df_to_mssql(product_model_df)


2024-06-07 18:08:21,180 - INFO - Connected to MongoDB database
2024-06-07 18:08:21,181 - INFO - {"message": "Waiting for suitable server to become available", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 6662ea25afbea97a38e960db, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('ac-wtfo0ij-shard-00-00.fnittpl.mongodb.net', 27017) server_type: Unknown, rtt: None>, <ServerDescription ('ac-wtfo0ij-shard-00-01.fnittpl.mongodb.net', 27017) server_type: Unknown, rtt: None>, <ServerDescription ('ac-wtfo0ij-shard-00-02.fnittpl.mongodb.net', 27017) server_type: Unknown, rtt: None>]>", "clientId": {"$oid": "6662ea25afbea97a38e960db"}, "remainingTimeMS": 30}
2024-06-07 18:08:22,950 - INFO - Connection to MongoDB closed.
2024-06-07 18:08:22,954 - INFO - Connected to MSSQL database
d:\Bi\BiDocument\HK1_2024\TLCN\eShopAnalysis\TLCN-eShopAnalysis\eShopAnalysis.ETL\eTLCartItemDW\jobs\load\loads.py:156: UserWarning: pandas only supports

                           ProductModelId  \
0    7F03EECB-218C-4E6D-B3F3-F6EE7B184CA3   
1    EEB4D276-E49E-4ED6-8A75-04231757FEDA   
2    C9856AFC-DB4D-41E0-8B80-128DE091E082   
3    9758B226-23E1-4CAA-8C2A-449D3375E294   
4    6185E4CE-1F17-4679-89B3-A040898EED0E   
..                                    ...   
142  FDA1738D-4CE4-4087-966B-731DE2F04BBA   
143  89E9FD7E-E4B2-44A6-91B1-D6E6A0AD8624   
144  CD2D577F-B79E-445D-BB22-0340B9A56268   
145  DEDE8A83-2436-43E8-9C82-81CA3EEEB12E   
146  2E9C501D-9A02-4360-87E3-3210D2AFEF8E   

                               ProductKey  \
0    8CD2075C-9574-4C4B-AA17-9E1759060773   
1    C001CBCB-AC0A-42BC-8067-22A89584D4E5   
2    21D32F7D-59F2-48EB-B526-954D66679271   
3    9682B122-93BB-4D57-AA3D-BE7FE41A9276   
4    BBFCFA77-730F-493E-8F9A-C1EF47A19EDA   
..                                    ...   
142  E0ADB939-BC77-4ACF-8D23-1720C657FDDB   
143  01091559-4883-4F19-8F25-D2F383654822   
144  A325ED0C-73D1-4857-B48D-E0F22FCCCEA2   
145  420C

In [9]:
from jobs.utils.datetime_utils import generate_date_df
from jobs.load.loads import load_date_df_to_mssql
date_df = generate_date_df(start='2023-12-01 00:00:00', end='2024-08-31 00:00:00')
load_date_df_to_mssql(date_df)

2024-06-07 18:08:33,467 - INFO - Connected to MSSQL database
d:\Bi\BiDocument\HK1_2024\TLCN\eShopAnalysis\TLCN-eShopAnalysis\eShopAnalysis.ETL\eTLCartItemDW\jobs\load\loads.py:183: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_date = pd.read_sql_query(query, mssql_cursor.connection)
2024-06-07 18:10:15,375 - INFO - Connection to MSSQL closed.


        DateKey        Date  Year  Quarter  Month  Day  Weekday WeekdayName
0  202312010218  2023-12-01  2023        4     12    1        4      Friday
1  202312010437  2023-12-01  2023        4     12    1        4      Friday
2  202312010656  2023-12-01  2023        4     12    1        4      Friday
3  202312010915  2023-12-01  2023        4     12    1        4      Friday
4  202312011134  2023-12-01  2023        4     12    1        4      Friday
5  202312011353  2023-12-01  2023        4     12    1        4      Friday
6  202312011612  2023-12-01  2023        4     12    1        4      Friday
7  202312011831  2023-12-01  2023        4     12    1        4      Friday
8  202312012050  2023-12-01  2023        4     12    1        4      Friday
9  202312012309  2023-12-01  2023        4     12    1        4      Friday


In [15]:
from jobs.extract.extracts import extract_mssql_cart_item_to_df
from jobs.transform.cart_items_orders_transform import transform_cart_item_df_to_cart_item_df_with_fkey
from jobs.load.loads import load_cart_item_df_to_mssql
cart_item_df = extract_mssql_cart_item_to_df()
cart_item_df_with_dim_keys = transform_cart_item_df_to_cart_item_df_with_fkey(cart_item_df)
load_cart_item_df_to_mssql(cart_item_df_with_dim_keys)

2024-06-07 19:11:45,484 - INFO - Connected to MSSQL database
d:\Bi\BiDocument\HK1_2024\TLCN\eShopAnalysis\TLCN-eShopAnalysis\eShopAnalysis.ETL\eTLCartItemDW\jobs\extract\extracts.py:197: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cart_item = pd.read_sql_query(query, mssql_data_src_cursor.connection)
2024-06-07 19:11:45,509 - INFO - Connection to MSSQL closed.
2024-06-07 19:11:45,570 - INFO - Connected to MSSQL database


                                 CartId                             ProductId  \
0  1ADE4F78-27A9-45D1-AD33-06E647082399  B7066432-1DD6-4B6B-B156-00DF502543B9   
1  A493E18C-4988-4999-9307-255DF8073A1D  B7066432-1DD6-4B6B-B156-00DF502543B9   
2  1013ABC4-97D0-4B93-809C-381388B4F557  B7066432-1DD6-4B6B-B156-00DF502543B9   
3  0B7FF0BA-2A37-4569-9009-6A82FFBF2B4B  B7066432-1DD6-4B6B-B156-00DF502543B9   
4  06B48463-07D1-4F7C-84E0-78323E67240F  B7066432-1DD6-4B6B-B156-00DF502543B9   

                         ProductModelId                           BusinessKey  \
0  8C0F7E64-437D-4424-A282-682D493AA8DB  CB85262E-3016-4AD7-B7A6-F08479B9E9C7   
1  8C0F7E64-437D-4424-A282-682D493AA8DB  CB85262E-3016-4AD7-B7A6-F08479B9E9C7   
2  8C0F7E64-437D-4424-A282-682D493AA8DB  CB85262E-3016-4AD7-B7A6-F08479B9E9C7   
3  8C0F7E64-437D-4424-A282-682D493AA8DB  CB85262E-3016-4AD7-B7A6-F08479B9E9C7   
4  8C0F7E64-437D-4424-A282-682D493AA8DB  CB85262E-3016-4AD7-B7A6-F08479B9E9C7   

  SaleItemId  IsOnSale  Sa

d:\Bi\BiDocument\HK1_2024\TLCN\eShopAnalysis\TLCN-eShopAnalysis\eShopAnalysis.ETL\eTLCartItemDW\jobs\load\loads.py:226: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cart_items = pd.read_sql_query(query, mssql_cursor.connection)
2024-06-07 19:11:46,519 - INFO - Connection to MSSQL closed.


                                    CartId  \
0     1ADE4F78-27A9-45D1-AD33-06E647082399   
1     A493E18C-4988-4999-9307-255DF8073A1D   
2     1013ABC4-97D0-4B93-809C-381388B4F557   
3     0B7FF0BA-2A37-4569-9009-6A82FFBF2B4B   
4     06B48463-07D1-4F7C-84E0-78323E67240F   
...                                    ...   
1390  D2053EE0-3EA4-4D38-AC28-40E03A30E625   
1391  12BCBD8E-7B48-4DD8-AAE2-622CAF62510C   
1392  C4BEF345-256A-4DAB-B17C-62CADDAB2843   
1393  B19441A4-BFA5-4695-B4F6-688DBB1D07D8   
1394  7259ED6A-B6E1-4C2E-9572-89AB1FD77967   

                            ProductModelId  \
0     8C0F7E64-437D-4424-A282-682D493AA8DB   
1     8C0F7E64-437D-4424-A282-682D493AA8DB   
2     8C0F7E64-437D-4424-A282-682D493AA8DB   
3     8C0F7E64-437D-4424-A282-682D493AA8DB   
4     8C0F7E64-437D-4424-A282-682D493AA8DB   
...                                    ...   
1390  F9C557EC-902D-49C3-9ADF-A290643F517F   
1391  F9C557EC-902D-49C3-9ADF-A290643F517F   
1392  F9C557EC-902D-49C3-9ADF-A29

#### OrderCart ETL

In [16]:
from jobs.utils.enum_utils import generate_payment_method_df, generate_discount_type_df, generate_order_status_df, generate_address_df
from jobs.load.loads import load_address_df_to_mssql, load_discount_type_df_to_mssql, load_order_status_df_to_mssql, load_payment_method_df_to_mssql
payment_method_df = generate_payment_method_df()
load_payment_method_df_to_mssql(payment_method_df)

2024-06-07 19:13:24,418 - INFO - Connected to MSSQL database
d:\Bi\BiDocument\HK1_2024\TLCN\eShopAnalysis\TLCN-eShopAnalysis\eShopAnalysis.ETL\eTLCartItemDW\jobs\load\loads.py:250: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_payment_methods = pd.read_sql_query(query, mssql_cursor.connection)
2024-06-07 19:13:24,428 - INFO - Connection to MSSQL closed.


   PaymentMethodId  MethodName
0                0         COD
1                1        Momo
2                2  CreditCard


In [17]:
discount_type_df = generate_discount_type_df()
load_discount_type_df_to_mssql(discount_type_df)

2024-06-07 19:13:28,006 - INFO - Connected to MSSQL database
d:\Bi\BiDocument\HK1_2024\TLCN\eShopAnalysis\TLCN-eShopAnalysis\eShopAnalysis.ETL\eTLCartItemDW\jobs\load\loads.py:273: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_discount_types = pd.read_sql_query(query, mssql_cursor.connection)
2024-06-07 19:13:28,018 - INFO - Connection to MSSQL closed.


   DiscountTypeId DiscountTypeName
0               0          ByValue
1               1        ByPercent
2               2       NoDiscount


In [18]:
order_status_df = generate_order_status_df()
load_order_status_df_to_mssql(order_status_df)

2024-06-07 19:13:32,704 - INFO - Connected to MSSQL database
d:\Bi\BiDocument\HK1_2024\TLCN\eShopAnalysis\TLCN-eShopAnalysis\eShopAnalysis.ETL\eTLCartItemDW\jobs\load\loads.py:296: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_order_statuses = pd.read_sql_query(query, mssql_cursor.connection)
2024-06-07 19:13:32,718 - INFO - Connection to MSSQL closed.


<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
   OrderStatusId        OrderStatusName
0              0           CreatedDraft
1              1  CustomerInfoConfirmed
2              2             Checkouted
3              3         StockConfirmed
4              4               Refunded
5              5              Cancelled
6              6              Completed


In [19]:
address_df = generate_address_df()
load_address_df_to_mssql(address_df)

2024-06-07 19:13:38,634 - INFO - Connected to MSSQL database
d:\Bi\BiDocument\HK1_2024\TLCN\eShopAnalysis\TLCN-eShopAnalysis\eShopAnalysis.ETL\eTLCartItemDW\jobs\load\loads.py:319: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_addresses = pd.read_sql_query(query, mssql_cursor.connection)
2024-06-07 19:13:38,646 - INFO - Connection to MSSQL closed.


   AddressId DistrictOrLocality
0          0              Quan1
1          1         QuanThuDuc
2          2      QuanBinhThanh
3          3        QuanTanBinh


In [1]:
from jobs.extract.extracts import extract_mssql_cart_order_to_df
from jobs.transform.cart_items_orders_transform import transform_cart_order_df_to_cart_order_df_with_fkey
from jobs.load.loads import load_cart_order_df_to_mssql

cart_order_df = extract_mssql_cart_order_to_df()
cart_order_df_with_dim_keys = transform_cart_order_df_to_cart_order_df_with_fkey(cart_order_df)
load_cart_order_df_to_mssql(cart_order_df_with_dim_keys)

2024-06-07 20:57:18,054 - INFO - Connected to MSSQL database
d:\Bi\BiDocument\HK1_2024\TLCN\eShopAnalysis\TLCN-eShopAnalysis\eShopAnalysis.ETL\eTLCartItemDW\jobs\extract\extracts.py:222: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cart_order = pd.read_sql_query(query, mssql_data_src_cursor.connection)
2024-06-07 20:57:18,093 - INFO - Connection to MSSQL closed.
2024-06-07 20:57:18,142 - INFO - Connected to MSSQL database


#### Recommend Cross Sell ETL

In [3]:
from jobs.extract.extracts import extract_mongo_interactions_to_df
from jobs.transform.product_interactions_transform import transform_product_user_interactions_df
interactions_df = extract_mongo_interactions_to_df()
interactions_df = transform_product_user_interactions_df(interactions_df)
interactions_df.to_csv('../recommendAndCrossSellApi/resources/product_user_interactions.csv', index=False)

2024-06-08 01:07:20,399 - INFO - Connected to MongoDB database
2024-06-08 01:07:20,400 - INFO - {"message": "Waiting for suitable server to become available", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 66634c5808a15cdcd23d6480, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None>]>", "clientId": {"$oid": "66634c5808a15cdcd23d6480"}, "remainingTimeMS": 30}
2024-06-08 01:07:20,428 - INFO - Connection to MongoDB closed.


In [4]:
from jobs.extract.extracts import extract_mssql_cart_item_transactions_to_df
from jobs.transform.cart_items_orders_transform import transform_cart_item_transactions_df
cart_item_transactions_df = extract_mssql_cart_item_transactions_to_df()
cart_item_transactions_df = transform_cart_item_transactions_df(cart_item_transactions_df)
cart_item_transactions_df.to_csv('../recommendAndCrossSellApi/resources/cart_item_transactions.csv', index=False)

2024-06-08 01:07:24,345 - INFO - Connected to MSSQL database
d:\Bi\BiDocument\HK1_2024\TLCN\eShopAnalysis\TLCN-eShopAnalysis\eShopAnalysis.ETL\eTLCartItemOrderDW\jobs\extract\extracts.py:251: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cart_item_transactions = pd.read_sql_query(query, mssql_data_src_cursor.connection)
2024-06-08 01:07:24,484 - INFO - Connection to MSSQL closed.
